In [1]:
!nvidia-smi

Tue May  3 22:06:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os, sys, random, shutil
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile
import pandas as pd
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
!unzip /content/drive/MyDrive/tobacco_yolo_format.zip

#### Cloning the Yolov5 git repo and installing the dependencies that are required by the Yolo model to train

In [4]:
!git clone https://github.com/ultralytics/yolov5.git
!sed -i 's/PyYAML>=5.3.1/PyYAML==5.4.1/g' /content/yolov5/requirements.txt
!pip install -qr '/content/yolov5/requirements.txt'

Cloning into 'yolov5'...
remote: Enumerating objects: 13163, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 13163 (delta 1), reused 1 (delta 0), pack-reused 13156
Receiving objects: 100% (13163/13163), 12.01 MiB | 8.39 MiB/s, done.
Resolving deltas: 100% (9150/9150), done.
     |████████████████████████████████| 636 kB 14.0 MB/s 


In [5]:
shutil.copyfile('/content/tobacco_data.yaml', '/content/yolov5/tobacco_data.yaml')
!sed -i 's/nc: 80/nc: 2/g' /content/yolov5/models/yolov5x.yaml
!cat /content/yolov5/tobacco_data.yaml

train: /content/tobacco_yolo_format/images/train
val: /content/tobacco_yolo_format/images/valid

nc: 1
names: ['DLSignature']


#### Using wandb to keep track of the training and valdiation metrics

In [ ]:
!pip install wandb
import wandb
wandb.init()

#### Running the train.py file to train the model for 200 epochs with weights saved every 50 epochs and train images converted to 640 by 640 pixels with a train batch size of 14. Batch size has been set so as to utilize the full GPU RAM that is available

In [ ]:
!python /content/yolov5/train.py --img 640 --batch 14 --epochs 400 \
    --data /content/yolov5/tobacco_data.yaml --cfg /content/yolov5/models/yolov5x.yaml --name yolo_train --save-period 50 --bbox_interval 50

wandb: Currently logged in as: vatsalmpatel (mlmodel). Use `wandb login --relogin` to force relogin
train: weights=yolov5/yolov5s.pt, cfg=/content/yolov5/models/yolov5x.yaml, data=/content/yolov5/tobacco_data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=14, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolo_train, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=50, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=50, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-173-gd4ea61e torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epo

In [ ]:
!python yolov5/detect.py --source "/content/data/signed_images_with_original_images" --img 640 --weights '/content/yolo_signature.pt' --line-thickness 2 

In [ ]:
# !python /content/yolov5/train.py --img 640 --batch 14 --epochs 3000 \
#     --data /content/yolov5/tobacco_data.yaml --cfg /content/yolov5/models/yolov5x.yaml --name Tobacco-run2 --save-period 50 --bbox_interval 50 --upload_dataset \
#     --weights /content/yolov5/runs/train/Tobacco-run/weights/best.pt